In [ ]:
# Import required libraries
import boto3
import pandas as pd
import json
import time
from IPython.display import display, Markdown

# Initialize AWS clients
region = 'us-west-2'  # Replace with your AWS region
bedrock = boto3.client('bedrock-runtime', region_name=region)
redshift_data = boto3.client('redshift-data', region_name=region)
session = boto3.Session()

In [ ]:
# Set Redshift cluster details
redshift_cluster_id = 'redshift-zero-etl-cluster'
database_name = 'dev'
db_user = 'admin'

In [ ]:
# Write the SQL query
sql_query = """
SELECT p.product_name, p.category, p.price, s.sale_amount, s.sale_date
FROM public.products p
JOIN public.sales s ON p.product_id = s.product_id
ORDER BY s.sale_date DESC
LIMIT 1;
"""

# Execute the query
response = redshift_data.execute_statement(
    ClusterIdentifier=redshift_cluster_id,
    Database=database_name,
    DbUser=db_user,
    Sql=sql_query
)

query_id = response['Id']

In [ ]:
# Wait for the query to complete
status = redshift_data.describe_statement(Id=query_id)['Status']
while status not in ('FINISHED', 'FAILED', 'ABORTED'):
    time.sleep(1)
    status = redshift_data.describe_statement(Id=query_id)['Status']

# Fetch the results
if status == 'FINISHED':
    result = redshift_data.get_statement_result(Id=query_id)
    records = result['Records']
else:
    raise Exception(f"Query failed with status: {status}")

In [ ]:
# Extract column names
column_info = result['ColumnMetadata']
columns = [col['name'] for col in column_info]

# Parse records
data = []
for record in records:
    row = []
    for field in record:
        value = list(field.values())[0]
        row.append(value)
    data.append(row)

# Create DataFrame
df = pd.DataFrame(data, columns=columns)
display(df)


In [ ]:
# Extract data for the prompt
product_info = df.iloc[0]

product_name = product_info['product_name']
category = product_info['category']
price = product_info['price']
sale_amount = product_info['sale_amount']
sale_date = product_info['sale_date']

# Create the prompt
prompt = f"""
You are a marketing expert.

Based on the following product information and recent sales data, generate a compelling marketing message.

Product Name: {product_name}
Category: {category}
Price: ${price}
Recent Sale Amount: ${sale_amount}
Sale Date: {sale_date}

The marketing message should highlight the product's features and appeal to potential customers.

Message:
"""

# Display the prompt
display(Markdown(f"**Prompt:**\n{prompt}"))

In [ ]:
# Set the model ID
model_id = 'amazon.titan-text-premier-v1:0'

# Invoke the model
response = bedrock.invoke_model(
    modelId=model_id,
    accept='application/json',
    contentType='application/json',
    body=json.dumps({
        "inputText": prompt,
        "parameters": {
            "maxTokens": 500,
            "temperature": 0.7,
            "topP": 0.9
        }
    })
)

# Parse the response
response_body = json.loads(response['body'].read())
generated_message = response_body['results'][0]['outputText']

# Display the generated message
display(Markdown(f"**Generated Marketing Message:**\n{generated_message}"))